# Создание и заполнение данных БД Postgre

In [1]:
import os
import json
import psycopg2
from psycopg2.extras import DictCursor


In [2]:
POSTGRESQL_HOST = '10.129.0.25'

In [3]:
!env | grep POST

POSTGRESQL_USER=reader
POSTGRESQL_PASSWORD=Miba2021


In [4]:
!psql -c 'create database test;'

CREATE DATABASE


In [5]:
%load_ext sql

In [6]:
%%sql postgresql:///test
    SELECT tablename AS table FROM pg_tables WHERE tablename !~ '^(pg_|sql_)'

0 rows affected.


table


## Примеры SQL запросов 
1. Представим запрос для создания таблицы users c двумя полями:
```
%%sql postgresql:///test
    CREATE TABLE users (
        login varchar(10) NOT NULL, 
        password varchar(100)
    );
```

2. Чтобы добавить запись в таблицу, небходимо выполнить следующий запрос:

```
%%sql postgresql:///test
   INSERT INTO users VALUES ('monit', 'querty')
```

3. Получить данные из таблицы:
```
%%sql postgresql:///test
   SELECT * FROM users
```

4. Удалить таблицу:
```
!psql -d test -c 'DROP TABLE users'
```

Считаем данные из файла datasets/aggrigation_logs_per_week.csv.

Выведем первую строку с заголовками. 

In [7]:
import csv

with open('datasets/aggrigation_logs_per_week.csv') as file:
    reader = csv.reader(file)
    headers = next(reader)
print (headers)

['courseid', 'userid', 'num_week', 's_all', 's_all_avg', 's_course_viewed', 's_course_viewed_avg', 's_q_attempt_viewed', 's_q_attempt_viewed_avg', 's_a_course_module_viewed', 's_a_course_module_viewed_avg', 's_a_submission_status_viewed', 's_a_submission_status_viewed_avg', 'NameR_Level', 'Name_vAtt', 'Depart', 'Name_OsnO', 'Name_FormOPril', 'LevelEd', 'Num_Sem', 'Kurs', 'Date_vAtt']


Вам предоставлен набор данных с логами (действиями) студентов на образовательном портале за весенний семестр (агрегация по каждой неделе) по отдельному электронному курсу.
- сourseid — уникальный идентификатор курса, дисциплины;
- userid — уникальный идентификатор студента (не используется в обучении);
- num_week — номер недели в году;
- s_all — количество всех событий на текущий момент;
- s_all_avg — среднее количество всех событий в неделю;
- s_course_viewed — количество просмотров курса;
- s_course_viewed_avg — среднее количество просмотров курса в неделю;
- s_q_attempt_viewed — количество просмотров теста;
- s_q_attempt_viewed_avg — среднее количество просмотров теста в неделю;
- s_a_course_module_viewed — количество просмотров модуля в курсе;
- s_a_course_module_viewed_avg — среднее количество просмотров модуля в курсе в неделю;
- s_a_submission_status_viewed — количество отправленных заданий на проверку;
- s_a_submission_status_viewed_avg — среднее количество ответов;
- namer_level — оценка за дисциплину;
- depart — номер кафедры;
- name_osno — основа обучения (имеет два значения: бюджет или контракт);
- name_formopril — форма обучения;
- leveled — уровень образования (имеет два значения: бакалавриат, магистратура);
- num_sem — номер семестра;
- kurs — номер курса учебной группы.

Создадим таблицу с логами студентов в тестовой БД

In [8]:
!psql -d test -c 'CREATE TABLE logs ( \
    courseid varchar(5), \
    userid varchar(5), \
    num_week smallint, \
    s_all smallint, \
    s_all_avg varchar(10), \
    s_course_viewed smallint, \
    s_course_viewed_avg varchar(10), \
    s_q_attempt_viewed smallint, \
    s_q_attempt_viewed_avg varchar(10), \
    s_a_course_module_viewed smallint, \
    s_a_course_module_viewed_avg varchar(10), \
    s_a_submission_status_viewed smallint, \
    s_a_submission_status_viewed_avg varchar(10), \
    NameR_Level varchar(20), \
    Name_vAtt varchar(20), \
    Depart varchar(5), \
    Name_OsnO smallint, \
    Name_FormOPril smallint, \
    LevelEd smallint, \
    Num_Sem smallint, \
    Kurs smallint, \
    Date_vAtt varchar(20)\
);'

CREATE TABLE


Добавим данные из файла aggrigation_logs_per_week.csv в созданную таблицу.


In [9]:
!psql -d test \
    -c "\copy logs FROM 'datasets/aggrigation_logs_per_week.csv' DELIMITER ',' CSV HEADER;"

COPY 414528


Выведем первые 10 записей.

In [11]:
%%sql postgresql:///test
    SELECT * FROM logs LIMIT 10

10 rows affected.


courseid,userid,num_week,s_all,s_all_avg,s_course_viewed,s_course_viewed_avg,s_q_attempt_viewed,s_q_attempt_viewed_avg,s_a_course_module_viewed,s_a_course_module_viewed_avg,s_a_submission_status_viewed,s_a_submission_status_viewed_avg,namer_level,name_vatt,depart,name_osno,name_formopril,leveled,num_sem,kurs,date_vatt
71262,34527,6,9,9,4,4,0,0,0,0,0,0,3,Экзамен,22,1,1,1,2,2,18.06.2022
71262,34527,7,0,"4,5",0,2,0,0,0,0,0,0,3,Экзамен,22,1,1,1,2,2,18.06.2022
71262,34527,8,0,3,0,"1,3333",0,0,0,0,0,0,3,Экзамен,22,1,1,1,2,2,18.06.2022
71262,34527,9,0,"2,25",0,1,0,0,0,0,0,0,3,Экзамен,22,1,1,1,2,2,18.06.2022
71262,34527,10,0,"1,8",0,"0,8",0,0,0,0,0,0,3,Экзамен,22,1,1,1,2,2,18.06.2022
71262,34527,11,1,"1,6667",1,"0,8333",0,0,0,0,0,0,3,Экзамен,22,1,1,1,2,2,18.06.2022
71262,34527,12,0,"1,4286",0,"0,7143",0,0,0,0,0,0,3,Экзамен,22,1,1,1,2,2,18.06.2022
71262,34527,13,0,"1,25",0,"0,625",0,0,0,0,0,0,3,Экзамен,22,1,1,1,2,2,18.06.2022
71262,34527,14,0,"1,1111",0,"0,5556",0,0,0,0,0,0,3,Экзамен,22,1,1,1,2,2,18.06.2022
71262,34527,15,1,"1,1",1,"0,6",0,0,0,0,0,0,3,Экзамен,22,1,1,1,2,2,18.06.2022


## Задание 1:

Измените данные вещественного типа, сейчас целая и дробная часть разделены запятой, замените ее на точку. 

Выведите первые 10 записей, чтобы проверить результат предобработки. 

In [12]:
%%sql postgresql:///test
UPDATE logs
SET 
    s_all_avg = REPLACE(s_all_avg, ',', '.')::NUMERIC,
    s_course_viewed_avg = REPLACE(s_course_viewed_avg, ',', '.')::NUMERIC,
    s_q_attempt_viewed_avg = REPLACE(s_q_attempt_viewed_avg, ',', '.')::NUMERIC,
    s_a_course_module_viewed_avg = REPLACE(s_a_course_module_viewed_avg, ',', '.')::NUMERIC,
    s_a_submission_status_viewed_avg = REPLACE(s_a_submission_status_viewed_avg, ',', '.')::NUMERIC;

414528 rows affected.


[]

In [16]:
%%sql postgresql:///test
SELECT * FROM logs LIMIT 10;

10 rows affected.


courseid,userid,num_week,s_all,s_all_avg,s_course_viewed,s_course_viewed_avg,s_q_attempt_viewed,s_q_attempt_viewed_avg,s_a_course_module_viewed,s_a_course_module_viewed_avg,s_a_submission_status_viewed,s_a_submission_status_viewed_avg,namer_level,name_vatt,depart,name_osno,name_formopril,leveled,num_sem,kurs,date_vatt
71559,25496,15,0,0.2,0,0.2,0,0,0,0,0,0,5,Экзамен,33,1,1,1,6,4,31.05.2022
71571,33389,6,38,38,17,17,0,0,2,2,2,2,4,Экзамен,20,1,1,1,2,2,21.06.2022
71571,33576,6,24,24,12,12,0,0,0,0,0,0,4,Экзамен,33,1,1,1,2,2,16.06.2022
71571,36145,25,0,0.2,0,0.1,0,0,0,0.05,0,0.05,3,Экзамен,20,1,1,1,2,2,17.06.2022
71571,36740,12,0,0,0,0,0,0,0,0,0,0,4,Экзамен,15,2,2,2,4,3,28.06.2022
71571,36835,13,0,0.25,0,0.125,0,0,0,0,0,0,4,Экзамен,20,2,3,1,2,2,24.06.2022
71596,29127,6,0,0,0,0,0,0,0,0,0,0,4,Экзамен,6,1,1,1,4,3,17.06.2022
71596,29213,15,3,11.3,1,2.6,0,0,1,3.6,1,2.6,5,Экзамен,6,1,1,1,4,3,17.06.2022
71596,29235,29,0,22.125,0,3.75,0,8.75,0,3,0,2.3333,5,Экзамен,6,1,1,1,4,3,27.06.2022
71596,29243,6,31,31,8,8,0,0,7,7,5,5,5,Экзамен,6,1,1,1,4,3,17.06.2022


## Задание 2: 

Выведите количество кафедр, за которыми закреплены курсы на портале.





In [17]:
%%sql postgresql:///test
SELECT COUNT(DISTINCT depart) AS department_count
FROM logs;

1 rows affected.


department_count
43


##  Задание 3:

Выведите сколько у каждой кафедры закреплено электронных курсов на портале. 

У какой кафедры больше всего курсов на портале?

In [18]:
%%sql postgresql:///test
SELECT
depart as departament_id,
COUNT(DISTINCT courseid) as course_count
FROM logs
GROUP BY depart
ORDER BY course_count DESC;

43 rows affected.


departament_id,course_count
11,53
19,42
23,42
8,40
15,36
4,35
9,35
21,33
14,33
3,33


## Задание 4:

Имеется информация о кодировании каждой кафедры:

| Id  | Name     | Id  | Name     | Id  | Name     | Id  | Name     |
|-----|---------|-----|---------|-----|---------|-----|---------|
| 1   | АиИИ    | 12  | ИиИБ    | 23  | ПОиД    | 34  | УСиБА    |
| 2   | АСУ     | 13  | ИТМ     | 24  | Психол. | 35  | Физики   |
| 3   | АЭПиМ   | 14  | ЛиП     | 25  | ПЭиБЖД  | 36  | Физкульт.|
| 4   | БИиИТ   | 15  | ЛиУТС   | 26  | РМПИ    | 37  | Химии    |
| 5   | ВИ      | 16  | ЛПиМ    | 27  | РЯОЯиМК | 38  | ХОМ      |
| 6   | ВТиП    | 17  | Менеджм.| 28  | СРиППО  | 39  | ЦДОМ     |
| 7   | ГМДиОПИ | 18  | МиТОДиМ | 29  | CC      | 40  | ЭиМЭ     |
| 8   | ГМиТТК  | 19  | МиХТ    | 30  | ТиЭС    | 41  | Эконом.  |
| 9   | ГМУиУП  | 20  | ПиСЗ    | 31  | ТОМ     | 42  | ЭПП      |
| 10  | Дизайна | 21  | ПиЭММО  | 32  | ТССА    | 43  | ЯиЛ      |
| 11  | ДиСО    | 22  | ПМиИ    | 33  | УиИС    |     |          |


Создайте отдельно таблицу со справочником (номер кафедры и название(аббревиатура) кафедры) departments.

Внесите данные в таблицу. 

In [51]:
%%sql postgresql:///test
CREATE TABLE departaments (
    id SERIAL PRIMARY KEY,
    name TEXT NOT NULL
)

(psycopg2.errors.DuplicateTable) relation "departaments" already exists

[SQL: CREATE TABLE departaments (
    id SERIAL PRIMARY KEY,
    name TEXT NOT NULL
)]
(Background on this error at: https://sqlalche.me/e/14/f405)


In [54]:
%%sql postgresql:///test

INSERT INTO departaments (id, name) VALUES
(1, 'АиИИ'), (2, 'АСУ'), (3, 'АЭПиМ'), (4, 'БИиИТ'), (5, 'ВИ'),
(6, 'ВТиП'), (7, 'ГМДиОПИ'), (8, 'ГМиТТК'), (9, 'ГМУиУП'), (10, 'Дизайна'),
(11, 'ДиСО'), (12, 'ИиИБ'), (13, 'ИТМ'), (14, 'ЛиП'), (15, 'ЛиУТС'),
(16, 'ЛПиМ'), (17, 'Менеджм.'), (18, 'МиТОДиМ'), (19, 'МиХТ'), (20, 'ПиСЗ'),
(21, 'ПиЭММО'), (22, 'ПМиИ'), (23, 'ПОиД'), (24, 'Психол.'), (25, 'ПЭиБЖД'),
(26, 'РМПИ'), (27, 'РЯОЯиМК'), (28, 'СРиППО'), (29, 'CC'), (30, 'ТиЭС'),
(31, 'ТОМ'), (32, 'ТССА'), (33, 'УиИС'), (34, 'УСиБА'), (35, 'Физики'),
(36, 'Физкульт.'), (37, 'Химии'), (38, 'ХОМ'), (39, 'ЦДОМ'), (40, 'ЭиМЭ'),
(41, 'Эконом.'), (42, 'ЭПП'), (43, 'ЯиЛ');

43 rows affected.


[]

In [55]:
%%sql postgresql:///test
SELECT * FROM departaments LIMIT 10;

10 rows affected.


id,name
1,АиИИ
2,АСУ
3,АЭПиМ
4,БИиИТ
5,ВИ
6,ВТиП
7,ГМДиОПИ
8,ГМиТТК
9,ГМУиУП
10,Дизайна


## Задание 5:

Выполните задание 3, но теперь нужно вывести не код кафедры, а ее название и количество курсов, которые за ней закреплены. 

У какой кафедры больше всего курсов?

In [59]:
%%sql postgresql:///test
SELECT d.name, COUNT(l.courseid) AS course_count
FROM departaments d
JOIN logs l ON d.id = CAST(l.depart AS INTEGER)
GROUP BY d.name
ORDER BY course_count DESC


43 rows affected.


name,course_count
МиХТ,25296
ДиСО,25176
ЛиП,19008
РМПИ,17856
ГМДиОПИ,16968
ТОМ,16704
БИиИТ,16176
ПОиД,14760
АЭПиМ,14232
ЛиУТС,13440


## Задание 6:

Ответьте на вопрос: существуют ли курсы, за которыми закреплено несколько кафедр? Если такие курсы есть, то выведите их количество.
Также выведите названия кафедр, которые совместно преподают один и тот же курс.




In [66]:
%%sql postgresql:///test
SELECT l.courseid, STRING_AGG(DISTINCT d.name, ', ') AS dept_names
FROM logs l
JOIN departaments d ON CAST(l.depart AS INTEGER) = d.id
GROUP BY l.courseid
HAVING COUNT(DISTINCT l.depart) > 1;

60 rows affected.


courseid,dept_names
71495,"ГМиТТК, ПиСЗ, УиИС"
71508,"ПиСЗ, УиИС"
71541,"ПиСЗ, УиИС"
71547,"ПиСЗ, УиИС"
71549,"ГМиТТК, ТиЭС"
71571,"ЛиУТС, ПиСЗ, УиИС"
71632,"CC, ВТиП"
71736,"ГМДиОПИ, ЭиМЭ"
71852,"АЭПиМ, ЭПП"
71857,"АЭПиМ, ЭПП"


## Задание 7:

Выведите количество студентов, которые получили 2, 3, 4, 5.

Пример вывода:

| namer_level |	count |
|-----|------|
|2 |	4 |
|3 |	3435 |
|4 | 	4676765|
|5 | 232 |


In [70]:
%%sql postgresql:///test
SELECT namer_level, COUNT(DISTINCT userid) AS student_count
FROM logs
WHERE CAST(namer_level AS INTEGER) IN (2, 3, 4, 5)
GROUP BY namer_level
ORDER BY namer_level;

4 rows affected.


namer_level,student_count
2,1069
3,1884
4,3243
5,3407


## Задание 8:

Выведите студента, который больше всех работает на портале (у него максимальное количество логов за вест период обучения).

In [71]:
%%sql postgresql:///test
SELECT userid, COUNT(*) AS log_count
FROM logs
GROUP BY userid
ORDER BY log_count DESC
LIMIT 1;

1 rows affected.


userid,log_count
35250,120


## Задание 9:

Выведите по каждой недели среднее количество всех событий на портале.

In [72]:
%%sql postgresql:///test
SELECT num_week, AVG(s_all) AS avg_s_all
FROM logs
GROUP BY num_week
ORDER BY num_week;

24 rows affected.


num_week,avg_s_all
6,13.7966072255673923
7,9.6161417322834646
8,8.0285433070866142
9,9.3932955071792497
10,8.2085456229735989
11,10.0220009263547939
12,9.3817160722556739
13,10.0140111162575266
14,9.8601783232978231
15,10.3536938397406207


## Задание 10: 

Выведите название кафедры, у которой больше всего отличников.

Отдельно выведите название кафедры, у которой больше всего двоечников. 

In [75]:
%%sql postgresql:///test
SELECT d.name, COUNT(DISTINCT l.userid) AS excellent_students
FROM logs l
JOIN departaments d ON CAST(l.depart AS INTEGER) = d.id
WHERE CAST(l.namer_level AS INTEGER) = 5
GROUP BY d.name
ORDER BY excellent_students DESC
LIMIT 1;

1 rows affected.


name,excellent_students
ДиСО,310


In [77]:
%%sql postgresql:///test
SELECT d.name, COUNT(DISTINCT l.userid) AS failing_students
FROM logs l
JOIN departaments d ON CAST(l.depart AS INTEGER) = d.id
WHERE CAST(l.namer_level AS INTEGER) = 2
GROUP BY d.name
ORDER BY failing_students DESC
LIMIT 1;

1 rows affected.


name,failing_students
Эконом.,72
